# Crash Data Insights

## Setup

In [19]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc
import matplotlib.pyplot as plt
import time
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

### Parameters and code maps

In [20]:
selected_counties = ['Delaware', 'Fairfield', 'Franklin', 'Knox', 'Licking',
                     'Madison', 'Marion', 'Morrow', 'Pickaway', 'Union', '10-County']

selected_crash_type = ['Rear End','Fixed Object','Sideswipe - Passing','Angle','Left Turn','Parked Vehicle','Animal','Backing','Right Turn','Head On','Pedestrian','Overturning','Pedacycles','Motorized', 'Non-motorized']


# Replace with your AGO credentials
username = "ogwynn_morpc"
password = "Cr33p3r11!!"

# Dictionary mapping county codes to county names
county_code_to_name = {
    'FRA': 'Franklin',
    'MAR': 'Marion',
    'DEL': 'Delaware',
    'UNI': 'Union',
    'LOG': 'Logan',
    'CRA': 'Crawford',
    'FAI': 'Fairfield',
    'KNO': 'Knox',
    'LOG': 'Logan',
    'LIC': 'Licking',
    'MAD': 'Madison',
    'MRW': 'Morrow',
    'PIC': 'Pickaway',
    'RIC': 'Richland',
    'ROS': 'Ross',
    'PER': 'Perry',
    'POR': 'Portage',
    'UNI': 'Union',
    'HAR': 'Hardin',
    'HAN': 'Hancock',
    'MOT': 'Montgomery',
    'SUM': 'Summit',
    'WYA': 'Wyandot',

    # Add more county codes and names as needed
}


## Data Retreival

### Pull crash data from AGO

In [21]:
import requests

def download_csv_from_ago(url, local_filename):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Write the content to a local CSV file
        with open(local_filename, 'wb') as file:
            file.write(response.content)
        
        print(f"CSV file downloaded successfully and saved as '{local_filename}'")
    
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while downloading the CSV: {e}")

# URL of the CSV file on AGO
url = 'https://morpc.maps.arcgis.com/sharing/rest/content/items/97a9bde6b3b24358b52ca86964714f76/data'

# Local filename to save the CSV
input1_path = 'inputData/MORPC_Crashes_2019-2023_GIS.csv'

# Call the function to download and save the CSV
download_csv_from_ago(url, input1_path)


CSV file downloaded successfully and saved as 'inputData/MORPC_Crashes_2019-2023_GIS.csv'


## Cleaning crash data

### Crash type summary

In [22]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv(input1_path)

# Summarize the 'CRASH_TYPE_CD' column by counting occurrences
crash_type_summary = df['CRASH_TYPE_CD'].value_counts()

# Display the summary
print("Summary of 'NLF_COUNTY_CD':")
print(crash_type_summary)

Summary of 'NLF_COUNTY_CD':
CRASH_TYPE_CD
Rear End                      49216
Fixed Object                  31842
Sideswipe - Passing           30963
Angle                         30806
Left Turn                     13670
Parked Vehicle                13090
Animal                        10178
Backing                        7572
Right Turn                     6179
Head On                        4197
Pedestrian                     2951
Other Non-Collision            2918
Overturning                    1351
Pedalcycles                    1257
Unknown                        1190
Other Object                    929
Sideswipe - Meeting             107
Train                            21
Other Non-Vehicle                 2
Falling From Or In Vehicle        1
Name: count, dtype: int64


### Grouping by location and crash type, sum fatalities and injuries

#### Sum for Non-motorized incidents

In [23]:
import pandas as pd

# Read the first CSV file into a DataFrame
df1 = pd.read_csv(input1_path)

# Group by 'Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD' and aggregate the specified columns
df2 = df1.groupby(['Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD']).agg({
    'ODPS_TOTAL_FATALITIES_NBR': 'sum',
    'INCAPAC_INJURIES_NBR': 'sum',
    'NON_INCAPAC_INJURIES_NBR': 'sum',
    'POSSIBLE_INJURIES_NBR': 'sum',
    'NO_INJURY_REPORTED_NBR': 'sum'
}).reset_index()

# Add a column for the count of rows summed
df2['CRASH_COUNT'] = df1.groupby(['Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD']).size().values

# Save the grouped DataFrame to a CSV file
input2_path = 'inputData/MORPC_Crashes_2019-2023_GIS_SUMMARY.csv'
df2.to_csv(input2_path, index=False)

# Classify crash types and create new aggregated rows for 'Motorized' and 'Non-motorized'
motorized_condition = ~df2['CRASH_TYPE_CD'].str.contains('Pedestrian|Pedalcycle', na=False)
non_motorized_condition = df2['CRASH_TYPE_CD'].str.contains('Pedestrian|Pedalcycle', na=False)

# Group for 'Motorized'
motorized = df2[motorized_condition].groupby(['Year', 'NLF_COUNTY_CD']).agg({
    'ODPS_TOTAL_FATALITIES_NBR': 'sum',
    'INCAPAC_INJURIES_NBR': 'sum',
    'NON_INCAPAC_INJURIES_NBR': 'sum',
    'POSSIBLE_INJURIES_NBR': 'sum',
    'NO_INJURY_REPORTED_NBR': 'sum',
    'CRASH_COUNT': 'sum'
}).reset_index()
motorized['CRASH_TYPE_CD'] = 'Motorized'

# Group for 'Non-motorized'
non_motorized = df2[non_motorized_condition].groupby(['Year', 'NLF_COUNTY_CD']).agg({
    'ODPS_TOTAL_FATALITIES_NBR': 'sum',
    'INCAPAC_INJURIES_NBR': 'sum',
    'NON_INCAPAC_INJURIES_NBR': 'sum',
    'POSSIBLE_INJURIES_NBR': 'sum',
    'NO_INJURY_REPORTED_NBR': 'sum',
    'CRASH_COUNT': 'sum'
}).reset_index()
non_motorized['CRASH_TYPE_CD'] = 'Non-motorized'

# Concatenate the new rows with the original DataFrame
df3 = pd.concat([df2, motorized, non_motorized], ignore_index=True)

# Save the final DataFrame to a CSV file
input3_path = 'inputData/MORPC_Crashes_2019-2023_GIS_SUMMARY_GROUPED.csv'
df3.to_csv(input3_path, index=False)

# Display the final DataFrame
df3.head()


,Year,NLF_COUNTY_CD,CRASH_TYPE_CD,ODPS_TOTAL_FATALITIES_NBR,INCAPAC_INJURIES_NBR,NON_INCAPAC_INJURIES_NBR,POSSIBLE_INJURIES_NBR,NO_INJURY_REPORTED_NBR,CRASH_COUNT
0,2019,CRA,Rear End,0,0,0,0,7,2
1,2019,DEL,Angle,2,15,184,140,1139,507
2,2019,DEL,Animal,0,1,6,16,419,322
3,2019,DEL,Backing,0,0,2,4,263,131
4,2019,DEL,Fixed Object,4,22,122,62,648,640


### Completing dataset

#### Enure row existis for each 'Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD' combination

In [24]:
import pandas as pd
import numpy as np

# Load the uploaded grouped casualty summary CSV
df = pd.read_csv(input3_path)

# Extract unique values for Year, NLF_COUNTY_CD, and CASUALTY_TYPE
years = df['Year'].unique()
counties = df['NLF_COUNTY_CD'].unique()
casualty_types = df['CRASH_TYPE_CD'].unique()

# Create a MultiIndex for all combinations of Year, NLF_COUNTY_CD, and CASUALTY_TYPE
full_index = pd.MultiIndex.from_product([years, counties, casualty_types], names=['Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD'])

# Reindex the DataFrame to ensure all combinations are present, filling missing rows with zeros
df4 = df.set_index(['Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD']).reindex(full_index, fill_value=0).reset_index()


# Save the resulting DataFrame to a new CSV file
input4_path = 'inputData/MORPC_Crashes_2019-2023_GIS_SUMMARY_GROUPED_FULL.csv'

df4.to_csv(input4_path, index=False)

# Display the resulting DataFrame
df4.head()

,Year,NLF_COUNTY_CD,CRASH_TYPE_CD,ODPS_TOTAL_FATALITIES_NBR,INCAPAC_INJURIES_NBR,NON_INCAPAC_INJURIES_NBR,POSSIBLE_INJURIES_NBR,NO_INJURY_REPORTED_NBR,CRASH_COUNT
0,2019,CRA,Rear End,0,0,0,0,7,2
1,2019,CRA,Angle,0,0,0,0,0,0
2,2019,CRA,Animal,0,0,0,0,0,0
3,2019,CRA,Backing,0,0,0,0,0,0
4,2019,CRA,Fixed Object,0,0,0,0,0,0


### County codes to names

In [25]:
import pandas as pd

# Load the local CSV file
local_filename = input4_path
df5 = pd.read_csv(local_filename)


# Replace 'NLF_COUNTY_CD' with county names
df5['NLF_COUNTY_CD'] = df5['NLF_COUNTY_CD'].map(county_code_to_name)

# Save the updated DataFrame to a new CSV file
input4_path = 'inputData/MORPC_Crashes_2019-2023_GIS_SUMMARY_GROUPED_FULL_1.csv'

df5.to_csv(input4_path, index=False)

print(f"Updated CSV with county names saved as '{input4_path}'")

df5.head()

Updated CSV with county names saved as 'inputData/MORPC_Crashes_2019-2023_GIS_SUMMARY_GROUPED_FULL_1.csv'


,Year,NLF_COUNTY_CD,CRASH_TYPE_CD,ODPS_TOTAL_FATALITIES_NBR,INCAPAC_INJURIES_NBR,NON_INCAPAC_INJURIES_NBR,POSSIBLE_INJURIES_NBR,NO_INJURY_REPORTED_NBR,CRASH_COUNT
0,2019,Crawford,Rear End,0,0,0,0,7,2
1,2019,Crawford,Angle,0,0,0,0,0,0
2,2019,Crawford,Animal,0,0,0,0,0,0
3,2019,Crawford,Backing,0,0,0,0,0,0
4,2019,Crawford,Fixed Object,0,0,0,0,0,0


### Summing for region 

In [26]:
# Filter the dataframe to include only the selected counties
df5_filtered = df5[df5['NLF_COUNTY_CD'].isin(selected_counties)]

# Group by the relevant columns and sum the 'POP' variable for selected counties
data2_1 = df5_filtered.groupby(
    ['Year', 'CRASH_TYPE_CD']
)[['ODPS_TOTAL_FATALITIES_NBR', 'INCAPAC_INJURIES_NBR', 'NON_INCAPAC_INJURIES_NBR', 'POSSIBLE_INJURIES_NBR', 'NO_INJURY_REPORTED_NBR', 'CRASH_COUNT']].sum().reset_index()

# Add a new column for the COUNTY identifier
data2_1['NLF_COUNTY_CD'] = '10-County'

# Rearrange columns to match the original dataset order
data2_1 = data2_1[['Year','NLF_COUNTY_CD','CRASH_TYPE_CD','CRASH_COUNT','ODPS_TOTAL_FATALITIES_NBR','INCAPAC_INJURIES_NBR','NON_INCAPAC_INJURIES_NBR','POSSIBLE_INJURIES_NBR','NO_INJURY_REPORTED_NBR']]

# Append the new '15County' entries to the filtered dataset
data3 = pd.concat([df5_filtered, data2_1], ignore_index=True)

# Save the data to CSV after appending the new entries
data3.to_csv('outputData/MORPC_Crashes_2019-2023_regional_summary_WIDE.csv', index=False)

# Reshape the data from wide to long format
data_long = pd.melt(data3, id_vars=['Year', 'NLF_COUNTY_CD', 'CRASH_TYPE_CD'],
                    var_name='Incident_Type', value_name='Incident_Count')

# Save the long-format data to CSV
data_long.to_csv('outputData/MORPC_Crashes_2019-2023_regional_summary_LONG.csv', index=False)

# Summarize the 'CRASH_TYPE_CD' column by counting occurrences
crash_type_summary = data3['NLF_COUNTY_CD'].value_counts()


# Display the summary
print("Summary of 'NLF_COUNTY_CD':")
print(crash_type_summary)

Summary of 'NLF_COUNTY_CD':
NLF_COUNTY_CD
Delaware     110
Fairfield    110
Franklin     110
Knox         110
Licking      110
Madison      110
Marion       110
Morrow       110
Pickaway     110
Union        110
10-County    110
Name: count, dtype: int64


#### Fatality plots

In [27]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# File path for input data
file_path = 'outputData/MORPC_Crashes_2019-2023_regional_summary_WIDE.csv'
df = pd.read_csv(file_path)

# Set global font size for consistent styling
plt.rcParams.update({'font.size': 18})

# Create output directory if it doesn't exist
output_dir = 'outputData/fatalityPlots'
os.makedirs(output_dir, exist_ok=True)

# Check if the file exists before proceeding
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    
    # Filter relevant crash types
    df = df[df['CRASH_TYPE_CD'].isin(['Motorized', 'Non-motorized'])]

    # List of selected counties
    selected_counties = df['NLF_COUNTY_CD'].unique()

    # Dictionary to store COUNTY to itemID mapping
    county_item_mapping = {}

    # Process data and generate plots for each selected county
    for county in selected_counties:
        county_data = df[df['NLF_COUNTY_CD'] == county]

        # Aggregate fatalities by year and crash type
        county_aggregated = county_data.groupby(['Year', 'CRASH_TYPE_CD'], as_index=False)['ODPS_TOTAL_FATALITIES_NBR'].sum()

        # Pivot data for visualization
        pivot_county_data = county_aggregated.pivot(index='Year', columns='CRASH_TYPE_CD', values='ODPS_TOTAL_FATALITIES_NBR').fillna(0)

        # Plotting the data with dual y-axes
        fig, ax1 = plt.subplots(figsize=(16, 9))

        # Plot Motorized fatalities
        ax1.plot(pivot_county_data.index, pivot_county_data['Motorized'], color='#365072', marker='o', label='Motorized Fatalities')
        ax1.set_ylabel('Motorized Fatalities', fontsize=18, color='#365072')
        ax1.tick_params(axis='y', labelcolor='#365072')
        ax1.set_xlabel('Year', fontsize=18)

        # Plot Non-motorized fatalities on secondary axis
        ax2 = ax1.twinx()
        ax2.plot(pivot_county_data.index, pivot_county_data['Non-motorized'], color='#0075BF', marker='s', label='Non-motorized Fatalities')
        ax2.set_ylabel('Non-motorized Fatalities', fontsize=18, color='#0075BF')
        ax2.tick_params(axis='y', labelcolor='#0075BF')

        # Adding data labels
        for i, year in enumerate(pivot_county_data.index):
            ax1.text(year, pivot_county_data['Motorized'][year], str(pivot_county_data['Motorized'][year]), ha='right', fontsize=14, color='#365072')
            ax2.text(year, pivot_county_data['Non-motorized'][year], str(pivot_county_data['Non-motorized'][year]), ha='right', fontsize=14, color='#0075BF')

        # Title and legend
        plt.title(f'Yearly Motorized vs. Non-Motorized Fatalities in {county} County (2019-2023)', fontsize=22, pad=20)
        fig.tight_layout()

        # Save the plot
        output_path = os.path.join(output_dir, f'{county}_fatality_trends_2019_2023.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()

        # Simulated mapping for demonstration
        county_item_mapping[county] = f"simulated_item_id_for_{county}"

    # Convert the dictionary to a DataFrame
    mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['NLF_COUNTY_CD', 'itemID'])
    mapping_df['insightType'] = 'fatalityTrends'
    mapping_df['TilesetID'] = 16

    # Save the mapping to a CSV file
    trends_mapping_file_path = 'AGO_County_Fatality_Trends_Mapping.csv'
    mapping_df.to_csv(trends_mapping_file_path, index=False)

    # Output the path of the generated CSV for confirmation
    print(f"CSV file saved at: {trends_mapping_file_path}")
else:
    print("Error: File not found.")

CSV file saved at: AGO_County_Fatality_Trends_Mapping.csv


## Uploading to AGO

In [28]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Directory where images are saved
output_dir = 'outputData/fatalityPlots'

# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Population proportion over time by race for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_df['TilesetID'] = 16
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)



Deleted existing item with title: 10-County fatality trends 2019 2023


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 10-County_fatality_trends_2019_2023.png with item ID: a6b2c6632d3f4c20af1e6bb042cf70ca and set it to public
Deleted existing item with title: Delaware fatality trends 2019 2023


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_fatality_trends_2019_2023.png with item ID: 8879636490194b9d96dca2668d0f578a and set it to public
Deleted existing item with title: Fairfield fatality trends 2019 2023


...

Uploaded Morrow_fatality_trends_2019_2023.png with item ID: f5d7a2456be545848d7b91c0329b24b8 and set it to public
Deleted existing item with title: Pickaway fatality trends 2019 2023


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Pickaway_fatality_trends_2019_2023.png with item ID: 1ae3ed6a303e43f2912402ed2d3fbbba and set it to public
Deleted existing item with title: Union fatality trends 2019 2023


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_fatality_trends_2019_2023.png with item ID: 6be6cd1807804883bd8d5cd918e5498c and set it to public


### Get catalog

In [29]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
csv_item = gis.content.get(catalog_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"CSV file downloaded successfully: {catalog_path}")

CSV file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


### Update Metadata Local

In [31]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path)

# Ensure consistent naming
mapping_df['COUNTY'] = mapping_df['COUNTY'].replace('10-County', '10County')

# Select only necessary columns from mapping_df
selected_mapping_df = mapping_df[['COUNTY', 'TilesetID', 'itemID']]

# Merge while keeping all columns from catalog_df
merged_df = catalog_df.merge(
    selected_mapping_df,
    left_on=['GeographyName', 'TilesetID'],
    right_on=['COUNTY', 'TilesetID'],
    how='left'
)

# Update 'ThumbnailURL' using 'itemID' where applicable
merged_df['ThumbnailURL'] = merged_df['itemID'].combine_first(merged_df['ThumbnailURL'])

# Drop unnecessary columns from the merged dataset
merged_df = merged_df.drop(columns=['COUNTY', 'itemID'], errors='ignore')

# Ensure the final DataFrame retains only original catalog_df columns (plus the updated ThumbnailURL)
expected_columns = list(catalog_df.columns)  # Retain all original columns
if 'ThumbnailURL' not in expected_columns:
    expected_columns.append('ThumbnailURL')  # Ensure ThumbnailURL is included

merged_df = merged_df[expected_columns]  # Reorder columns to match the original catalog

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
merged_df.to_excel(updated_catalog_path, index=False)

print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


## Push changes to AGO

In [32]:
from arcgis.gis import GIS

# Title or Item ID of the existing XLSX file in AGO
item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
local_xlsx_path = updated_catalog_path

# --- Search for the Existing Item ---
items = gis.content.search(item_id, item_type="Microsoft Excel")
if not items:
    print("Error: No matching XLSX file found.")
else:
    item = items[0]  # Assuming the first match is correct
    print(f"Found item: {item.title} ({item.id})")

    # --- Update the Item with the New File ---
    item.update({}, local_xlsx_path)

    print(f"Successfully replaced {item.title} with the new file.")


Found item: morpc_insights_catalog (c396b9d0db724d1f870fa47f1d1dc1ce)
Successfully replaced morpc_insights_catalog with the new file.
